## Import

In [1]:
import os
import yaml
import warnings
from typing import List

from utils import get_openai_api_key
from utils_data import add_slot, is_slot_conflict, remove_slot, access_timetable, access_timetable_with_date

import crewai
from crewai import Agent, Task, Crew
from pydantic import BaseModel, Field
from crewai.tools import BaseTool


warnings.filterwarnings('ignore')

In [2]:
#TIMETABLE = {}

"""{'06/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}],
 '13/10/2025': [{'weekday': 'Monday',
   'startTime': '17:00',
   'endTime': '18:00',
   'title': 'Meeting with Boss'}]}"""

"""
    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],
    "anotherdate": [...], 
"""

'\n    "date": [{"startTime", "endTime", "title"}, {...}, {...}, ...],\n    "anotherdate": [...], \n'

#### Get api key

In [3]:
openai_api_key = get_openai_api_key()

## Setup configs

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Pydantic models

In [5]:
class EventSlot(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")
    startTime: str = Field(..., description="Start time of the slot. The format is: HH:MM")
    endTime: str = Field(..., description="End time of the slot. The format is: HH:MM")
    title: str = Field(..., description="Short title to summarize the event's essential information.")

class MultipleSlots(BaseModel):
    slots: List[EventSlot] = Field(..., description="List of event slots")

## Tools

In [6]:
from datetime import date
from typing import Type


from datetime import datetime

class InputDate(BaseModel):
    date: str = Field(..., description="The date of the event. The format is: DD/MM/YYYY")
    weekday: str = Field(..., description="The date's week day.")

class ValidDateTool(BaseTool):
    name: str = "Valid Date Tool"
    description: str = "Tool that checks if the date and week day are valid"
    args_schema: Type[BaseModel] = InputDate

    def _run(self, date: str, weekday: str):
        try:
            datetime.strptime(date, "%d/%m/%Y")
        except ValueError:
            return f"Error: {str(e)}. Please provide a valid date."
        
        try:
            date_obj = datetime.strptime(date, "%d/%m/%Y")
            correct_weekday = date_obj.strftime("%A")  # Full name (e.g., "Monday")
            correct_weekday.lower().startswith(weekday.lower())
            return "Valid date"
        except ValueError as e:
            return f"Error: {str(e)}. The week day does not corresponds to the date"
        
class AddSlot(BaseTool):
    name: str = "Add Slot Tool"
    description: str = "Tool that adds slots to the timetable using these informations: date, weekday, start time, end time and the title of the event."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        for eventSlot in slots:
            res = add_slot(eventSlot)
            if res != "SUCCESS":
                return res
        return "Successfully added slot(s)"
    
class RemoveSlot(BaseTool):
    name: str = "Remove Slot Tool"
    description: str = "Tool that removes slots to the timetable using these informations: date, weekday, start time, end time and the title of the event.\n" \
    "Do not call this tool again if already successful."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        res = ""
        for eventSlot in slots:
            res += remove_slot(eventSlot) + "\n"
        return res

class AccessTimetable(BaseTool):
    name: str = "Access Timetable"
    description: str = "Tool that enables access to slots on a specific date in dict (format of date DD/MM/YYYY)"
    
    def _run(self, date_input: str):
        return access_timetable_with_date(date_input)
            
    
class CheckConflict(BaseTool):
    name: str = "Check Conflict Tool"
    description: str = "Tool that checks if there is a conflict between an event and an already setup event in the timetable. " \
    "Returns the list of events where there is a conflict with already setup events in the timetable."
    args_schema: Type[BaseModel] = MultipleSlots

    def _run(self, slots: MultipleSlots):
        conflicts = []
        for slot in slots:
            res = is_slot_conflict(slot)
            if res == "There is a conflict":
                conflicts.append(slot)
        return conflicts


## Agents

In [7]:
converter = Agent(
    config=agents_config["converter"],
    inject_date=True,
    date_format="%A, %B %d, %Y",
    tools=[ValidDateTool()])

assistant = Agent(
    config=agents_config["assistant"],
    tools=[AddSlot(), CheckConflict(), RemoveSlot(), AccessTimetable()]
)

## Tasks

In [8]:
conversion = Task(
    config=tasks_config["conversion"],
    agent=converter,
    output_pydantic=MultipleSlots
)

manage_timetable = Task(
    config=tasks_config["manage_timetable"],
    agent=assistant
)

## Crew

In [ ]:
crew = Crew(
    agents=[converter, assistant],
    tasks=[conversion, manage_timetable],
    verbose=False
)

## Execution

In [10]:
inputs_array = [{"event": "I have a meeting with my boss for the next two mondays at 5pm for an hour (6th and 13th october)"},
                {"event": "Can you delete the meeting on the 6th october"}]

In [ ]:
result = crew.kickoff_for_each(inputs=inputs_array)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: aabeb2c7-a9ba-4dc9-9741-dcef89b3a0f1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Event summarizer                                                                                        │
│                                                                                                                 │
│  Task: Gather the most crucial information about the event: I have a meeting with my boss for the next two      │
│  mondays at 5pm for an hour (6th and 13th october). You should gather the following if there if:                │
│      1. Date in format DD/MM/YYYY                                                                               │
│      2. Start time in format HH:MM                                                                              │
│      3. End time in format HH:MM                                                                                │
│      4. Event title, generate if not explicit                                                                   │
│  Make sure you are sending a valid date.                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│  Current Date: Tuesday, September 30, 2025                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Event summarizer                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to verify the dates 6th and 13th October for validity since the current date is       │
│  September 30, 2025.                                                                                            │
│                                                                                                                 │
│  Using Tool: Valid Date Tool                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"date\": \"06/10/2025\", \"weekday\": \"Monday\"}"                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Valid date                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Slot deleted. Ask customer to ask to delete again by giving correct information if not really deleted.         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable assistant                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The meeting titled "Meeting with boss" scheduled on the 6th of October from 17:00 to 18:00 has been deleted    │
│  from the timetable.                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: be631b8d-8e9c-4f58-bb69-55f0506e3b18                                                                     │
│  Agent: Timetable assistant                                                                                     │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 10ced502-da6c-482d-9360-5e5d852b1c03                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The meeting titled "Meeting with boss" scheduled on the 6th of October from 17:00 to 18:00 has   │
│  been deleted from the timetable.                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

TypeError: object list can't be used in 'await' expression

In [ ]:
#result = await crew.kickoff_async(inputs={"event": "Can you delete the meeting on the 6th october"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 045a85bb-27dc-495b-b86f-31bf5c12a03d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Event summarizer                                                                                        │
│                                                                                                                 │
│  Task: Gather the most crucial information about the event: Can you delete the meeting on the 6th october. You  │
│  should gather the following if there if:                                                                       │
│      1. Date in format DD/MM/YYYY                                                                               │
│      2. Start time in format HH:MM                                                                              │
│      3. End time in format HH:MM                                                                                │
│      4. Event title, generate if not explicit                                                                   │
│  Make sure you are sending a valid date.                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│  Current Date: Tuesday, September 30, 2025                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Event summarizer                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to check if the date "6th October" is valid and determine the correct format for the  │
│  date and any associated weekday.                                                                               │
│                                                                                                                 │
│  Using Tool: Valid Date Tool                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"date\": \"06/10/2025\", \"weekday\": \"Monday\"}"                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Valid date                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:          │
│                                                                                                                 │
│  Tool Name: Valid Date Tool                                                                                     │
│  Tool Arguments: {'date': {'description': 'The date of the event. The format is: DD/MM/YYYY', 'type': 'str'},   │
│  'weekday': {'description': "The date's week day.", 'type': 'str'}}                                             │
│  Tool Description: Tool that checks if the date and week day are valid                                          │
│                                                                                                                 │
│  IMPORTANT: Use the following format in your response:                                                          │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, only one name of [Valid Date Tool], just the name, exactly as it's written.        │
│  Action Input: the input to the action, just a simple JSON object, enclosed in curly braces, using " to wrap    │
│  keys and values.                                                                                               │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│                                                                                                                 │
│  Once all necessary information is gathered, return the following format:                                       │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now know the final answer                                                                           │
│  Final Answer: the final answer to the original input question                                                  │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"date_input\": \"06/10/2025\"}"                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'Status': 'No events planned'}                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Timetable assistant                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  There are no slots to delete on the 6th of October 2025 as the timetable indicates no events planned for that  │
│  date.                                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b3d5fe02-a3c4-40d4-bc92-5c6d965162b7                                                                     │
│  Agent: Timetable assistant                                                                                     │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 045a85bb-27dc-495b-b86f-31bf5c12a03d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: There are no slots to delete on the 6th of October 2025 as the timetable indicates no events     │
│  planned for that date.                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
access_timetable_with_date("06/10/2025")

loading
TIMETABLE: {'06/10/2025': [{'weekday': 'Monday', 'startTime': '17:00', 'endTime': '18:00', 'title': 'Meeting with Boss'}], '13/10/2025': [{'weekday': 'Monday', 'startTime': '17:00', 'endTime': '18:00', 'title': 'Meeting with Boss'}]}


[{'weekday': 'Monday',
  'startTime': '17:00',
  'endTime': '18:00',
  'title': 'Meeting with Boss'}]

In [ ]:
# result.pydantic.model_dump()["slots"]

In [ ]:
result

CrewOutput(raw='There are no slots to delete on the 6th of October 2025 as the timetable indicates no events planned for that date.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Gather the most crucial information about the event: Can you delete the meeting on the 6th october. You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in format HH:MM\n    4. Event title, generate if not explicit\nMake sure you are sending a valid date.\n\n\nCurrent Date: Tuesday, September 30, 2025', name='Gather the most crucial information about the event: Can you delete the meeting on the 6th october. You should gather the following if there if:\n    1. Date in format DD/MM/YYYY\n    2. Start time in format HH:MM\n    3. End time in format HH:MM\n    4. Event title, generate if not explicit\nMake sure you are sending a valid date.\n\n\nCurrent Date: Tuesday, September 30, 2025', expected_output='A compre

In [ ]:
# result.pydantic.model_dump()

In [ ]:
access = AccessTimetable()

access._run("06/10/2025")

loading
TIMETABLE: {'06/10/2025': [{'weekday': 'Monday', 'startTime': '17:00', 'endTime': '18:00', 'title': 'Meeting with Boss'}], '13/10/2025': [{'weekday': 'Monday', 'startTime': '17:00', 'endTime': '18:00', 'title': 'Meeting with Boss'}]}


[{'weekday': 'Monday',
  'startTime': '17:00',
  'endTime': '18:00',
  'title': 'Meeting with Boss'}]

## Cost

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0023


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,15279,14330,5504,949,13


In [ ]:
1

1